In [1]:
import pyarrow.parquet as pq
import re

In [2]:
df_items = pq.read_table('../data2/dfitemsOpen_compr.parquet').to_pandas()
df_items = df_items.dropna(how='all')
df_items = df_items.drop(['0'], axis=1)

In [3]:
# Elimino las filas donde 'item_id', 'user_id', 'playtime_forever' son vacios
df_items = df_items.dropna(subset=['item_id', 'user_id', 'playtime_forever'])

# Elimino duplicados
df_items = df_items.drop_duplicates()

In [4]:
# Elimino columnas no relevantes
columnas = ['steam_id', 'user_url']
df_items = df_items.drop(columnas, axis=1)

In [5]:
# Elimino los caracteres especiales de la columna 'item_name'

df_items['item_name'] = df_items['item_name'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))
df_items['item_name'] = df_items['item_name'].apply(lambda x: re.sub(r'\s+', ' ', x))

In [6]:
# Convierto todas las palabras de la columna 'item_name' en minúsculas
df_items['item_name'] = df_items['item_name'].str.lower()

In [10]:
# Convierto 'item_id' a integer
df_items['item_id'] = df_items['item_id'].astype(int)

In [14]:
df_items.head(2)

,user_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,10,counter strike,6.0,0.0
1,76561197970982479,277,20,team fortress classic,0.0,0.0


In [15]:
# Guardo el dataset procesado para su posterior uso
df_items.to_parquet('../data2/df_items_limpio_compr.parquet')